In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
data = pd.read_csv('./credit_train.csv', encoding = 'cp1251', sep = ';')

In [3]:
data.shape

(170746, 15)

In [4]:
data.head()

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


__Список исходных переменных включает в себя:__
- категориальный предиктор *Идентификационный номер __[client_id]__*
- категориальный предиктор *Пол __[gender]__*
- количественный предиктор *Возраст __[age]__*
- категориальный предиктор *Семейный статус __[marital_status]__*
- категориальный предиктор *Сфера занятости __[job_position]__*
- количественный предиктор *Сумма кредита __[credit_sum]__*
- количественный предиктор *Срок кредитования __[credit_month]__*
- количественный предиктор *Внутренняя скоринговая оценка __[score_shk]__*
- категориальный предиктор *Образование __[education]__*
- категориальный предиктор *Идентификационный номер тарифа __[tariff_id]__*
- количественный предиктор *Месячный заработок __[monthly_income]__*
- количественный предиктор *Количество кредитов у клиента __[credit_count]__*
- количественный предиктор *Количество просроченных кредитов у клиента __[overdue_credit_count]__*
- категориальная зависимая переменная *Факт открытия кредитного счета в данном банке __[open_account_flg]__*

__ПЛАН ПРЕДВАРИТЕЛЬНОЙ ПОДГОТОВКИ ДАННЫХ__

__До разбиения набора данных на обучающую и контрольную выборки__
- Удаление очевидных бесполезных переменных (переменных у которых количество категорий совпадает с количеством наблюдений, или переменных с одним уникальным значением;
- Преобразование типов данных;
- Нормализация строковых значений (удаление лишних символов, приведение к одному и тому же регистру);
- Обработка дублирующих наблюдений;
- Обработка редких категорий, которую можно выполнить до разбиения на обучающую и контрольную выборки;
- Импутация пропусков, которую можно выполнить до разбиения на обучающую и контрольную выборки;
- Конструирование признаков, которое можно выполнить до разбиения на обучающую и контрольную выборки;

__После разбиения набора данных на обучающую и контрольную выборки__
- Выполнение преобразований, максимизирующих нормальность;
- Обработка редких категорий, которую можно выполнить только после разбиения на обучающую и контрольную выборки;
- Импутация пропусков, которую можно выполнить только после разбиения на обучающую и контрольную выборки;
- Конструирование признаков, которое можно выполнить только после разбиения на обучающую и контрольную выборки;
- Стандартизация, дамми-кодирование

In [5]:
# для начала удалим идентификационную переменную client_id. Эта переменная имеет столько же уникальных значений, сколько
# у нас имеется наблюдений

data.drop('client_id', axis = 1, inplace = True)
data.head(5)

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


In [6]:
# выведем уникальные значения по переменной living_region с помощью метода .unique()
data['living_region'].unique()

array(['КРАСНОДАРСКИЙ КРАЙ', 'МОСКВА', 'ОБЛ САРАТОВСКАЯ',
       'ОБЛ ВОЛГОГРАДСКАЯ', 'ЧЕЛЯБИНСКАЯ ОБЛАСТЬ', 'СТАВРОПОЛЬСКИЙ КРАЙ',
       'ОБЛ НИЖЕГОРОДСКАЯ', 'МОСКОВСКАЯ ОБЛ',
       'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА', 'КРАЙ СТАВРОПОЛЬСКИЙ',
       'САНКТ-ПЕТЕРБУРГ', 'РЕСП. БАШКОРТОСТАН', 'ОБЛ АРХАНГЕЛЬСКАЯ',
       'ХАНТЫ-МАНСИЙСКИЙ АО', 'РЕСП БАШКОРТОСТАН', 'ПЕРМСКИЙ КРАЙ',
       'РЕСП КАРАЧАЕВО-ЧЕРКЕССКАЯ', 'САРАТОВСКАЯ ОБЛ', 'ОБЛ КАЛУЖСКАЯ',
       'ОБЛ ВОЛОГОДСКАЯ', 'РОСТОВСКАЯ ОБЛ', 'УДМУРТСКАЯ РЕСП',
       'ОБЛ ИРКУТСКАЯ', 'ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ', 'ОБЛ МОСКОВСКАЯ',
       'ОБЛ ТЮМЕНСКАЯ', 'ОБЛ БЕЛГОРОДСКАЯ', 'РОСТОВСКАЯ ОБЛАСТЬ',
       'ОБЛ КОСТРОМСКАЯ', 'РЕСП ХАКАСИЯ', 'РЕСПУБЛИКА ТАТАРСТАН',
       'ИРКУТСКАЯ ОБЛАСТЬ', 'ОБЛ СВЕРДЛОВСКАЯ', 'ОБЛ ПСКОВСКАЯ',
       'КРАЙ ЗАБАЙКАЛЬСКИЙ', 'СВЕРДЛОВСКАЯ ОБЛ', 'ОБЛ ОРЕНБУРГСКАЯ',
       'ОБЛ ВОРОНЕЖСКАЯ', 'ОБЛ АСТРАХАНСКАЯ', 'ОБЛ НОВОСИБИРСКАЯ',
       'ОБЛ ЧЕЛЯБИНСКАЯ', 'ОРЕНБУРГСКАЯ ОБЛ', 'СВЕРДЛОВСКАЯ ОБЛАСТЬ'

In [7]:
# теперь с помощью метода .nunique() выведем количество уникальных значений переменной
data['living_region'].nunique()

301

In [8]:
# один и тот же регион может быть записан по разному, кроме того присутствуют иные названия не имеющие отношения к регионам

# увеличим максимальное количество отбражаемых строк
pd.options.display.max_rows = 310

In [9]:
# уникальные значения переменной living_region записываем в отдельный объект regions 
regions = data['living_region'].unique()

In [10]:
# теперь создаем серию у которой в качестве индексных меток и значений будут выступать уникальные значения переменной 
# living_region записанные в regions 
regions = pd.Series(data = regions, index = regions, name = 'regions')
regions

КРАСНОДАРСКИЙ КРАЙ                                                КРАСНОДАРСКИЙ КРАЙ
МОСКВА                                                                        МОСКВА
ОБЛ САРАТОВСКАЯ                                                      ОБЛ САРАТОВСКАЯ
ОБЛ ВОЛГОГРАДСКАЯ                                                  ОБЛ ВОЛГОГРАДСКАЯ
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ                                              ЧЕЛЯБИНСКАЯ ОБЛАСТЬ
СТАВРОПОЛЬСКИЙ КРАЙ                                              СТАВРОПОЛЬСКИЙ КРАЙ
ОБЛ НИЖЕГОРОДСКАЯ                                                  ОБЛ НИЖЕГОРОДСКАЯ
МОСКОВСКАЯ ОБЛ                                                        МОСКОВСКАЯ ОБЛ
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА    ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА
КРАЙ СТАВРОПОЛЬСКИЙ                                              КРАЙ СТАВРОПОЛЬСКИЙ
САНКТ-ПЕТЕРБУРГ                                                      САНКТ-ПЕТЕРБУРГ
РЕСП. БАШКОРТОСТАН                                               

In [11]:
# создаем список стоп слов - слов которые не несут никакой смысловой нагрузки и которые можно проигнорировать
stopwrds = set(['ОБЛ', 'ОБЛАСТЬ', 'РЕСП', 'РЕСПУБЛИКА',
                'КРАЙ', 'Г', 'АО', 'АОБЛ', 'АВТОНОМНАЯ'])

In [12]:
# пишем функцию для предобработки значений серии
def clean_region(x):
    x = re.sub('[.,]+', ' ', str(x))
    wrds = x.split(' ')
    wrds_new = []
    for w in wrds:
        if not w in stopwrds:
            wrds_new.append(w)
    x = ''.join(wrds_new)
    return x    

In [13]:
# применяем функцию к нашей серии
regions = regions.map(clean_region)
regions

КРАСНОДАРСКИЙ КРАЙ                                                 КРАСНОДАРСКИЙ
МОСКВА                                                                    МОСКВА
ОБЛ САРАТОВСКАЯ                                                      САРАТОВСКАЯ
ОБЛ ВОЛГОГРАДСКАЯ                                                  ВОЛГОГРАДСКАЯ
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ                                                  ЧЕЛЯБИНСКАЯ
СТАВРОПОЛЬСКИЙ КРАЙ                                               СТАВРОПОЛЬСКИЙ
ОБЛ НИЖЕГОРОДСКАЯ                                                  НИЖЕГОРОДСКАЯ
МОСКОВСКАЯ ОБЛ                                                        МОСКОВСКАЯ
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА    ХАНТЫ-МАНСИЙСКИЙАВТОНОМНЫЙОКРУГ-ЮГРА
КРАЙ СТАВРОПОЛЬСКИЙ                                               СТАВРОПОЛЬСКИЙ
САНКТ-ПЕТЕРБУРГ                                                  САНКТ-ПЕТЕРБУРГ
РЕСП. БАШКОРТОСТАН                                                  БАШКОРТОСТАН
ОБЛ АРХАНГЕЛЬСКАЯ           

In [14]:
# вносим финальные корректировки в regions 
regions['ЧУКОТСКИЙ АО'] = 'ЧУКОТСКИЙ'
regions['ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions['РЕСП ЧУВАШСКАЯ - ЧУВАШИЯ'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШСКАЯ - ЧУВАШИЯ РЕСП'] = 'ЧУВАШСКАЯ'
regions['ЧУВАШИЯ ЧУВАШСКАЯ РЕСПУБЛИКА -'] = 'ЧУВАШСКАЯ'
regions['РЕСПУБЛИКАТАТАРСТАН'] = 'ТАТАРСТАН'
regions['ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ'] = 'МОСКОВСКАЯ'
regions['ПЕРМСКАЯ ОБЛ'] = 'ПЕРМСКИЙ'
regions['ОРЁЛ'] = 'ОРЛОВСКАЯ'
regions['Г.ОДИНЦОВО МОСКОВСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions['МЫТИЩИНСКИЙ Р-Н'] = 'МОСКОВСКАЯ'
regions['МОСКОВСКИЙ П'] = 'МОСКОВСКАЯ'
regions['КАМЧАТСКАЯ ОБЛАСТЬ'] = 'КАМЧАТСКИЙ'
regions['ДАЛЬНИЙ ВОСТОК'] = 'МОСКОВСКАЯ'
regions['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions['ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н'] = 'ВЛАДИМИРСКАЯ'
regions['ГОРЬКОВСКАЯ ОБЛ'] = 'НИЖЕГОРОДСКАЯ'
regions['ЭВЕНКИЙСКИЙ АО'] = 'КРАСНОЯРСКИЙ'
regions['ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['АО ХАНТЫ-МАНСИЙСКИЙ-ЮГРА'] = 'ХАНТЫ-МАНСИЙСКИЙ'
regions['СЕВ. ОСЕТИЯ - АЛАНИЯ'] = 'СЕВЕРНАЯОСЕТИЯ-АЛАНИЯ'
regions['РЕСП. САХА (ЯКУТИЯ)'] = 'САХА/ЯКУТИЯ/'
regions['РЕСПУБЛИКА САХА'] = 'САХА/ЯКУТИЯ/'
regions['ДАЛЬНИЙВОСТОК'] = 'МОСКОВСКАЯ'
regions['САХА'] = 'САХА/ЯКУТИЯ/'
regions['98'] = 'САНКТ-ПЕТЕРБУРГ'
regions['74'] = 'ЧЕЛЯБИНСКАЯ'
regions['РОССИЯ'] = 'МОСКОВСКАЯ'
regions['МОСКВОСКАЯ'] = 'МОСКОВСКАЯ'
regions['МОСКВОСКАЯ ОБЛ'] = 'МОСКОВСКАЯ'
regions['ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions['Г. ЧЕЛЯБИНСК'] = 'ЧЕЛЯБИНСКАЯ'
regions['БРЯНСКИЙ'] = 'БРЯНСКАЯ'

In [15]:
# вновь смотрим серию
regions 

КРАСНОДАРСКИЙ КРАЙ                                  КРАСНОДАРСКИЙ
МОСКВА                                                     МОСКВА
ОБЛ САРАТОВСКАЯ                                       САРАТОВСКАЯ
ОБЛ ВОЛГОГРАДСКАЯ                                   ВОЛГОГРАДСКАЯ
ЧЕЛЯБИНСКАЯ ОБЛАСТЬ                                   ЧЕЛЯБИНСКАЯ
СТАВРОПОЛЬСКИЙ КРАЙ                                СТАВРОПОЛЬСКИЙ
ОБЛ НИЖЕГОРОДСКАЯ                                   НИЖЕГОРОДСКАЯ
МОСКОВСКАЯ ОБЛ                                         МОСКОВСКАЯ
ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА         ХАНТЫ-МАНСИЙСКИЙ
КРАЙ СТАВРОПОЛЬСКИЙ                                СТАВРОПОЛЬСКИЙ
САНКТ-ПЕТЕРБУРГ                                   САНКТ-ПЕТЕРБУРГ
РЕСП. БАШКОРТОСТАН                                   БАШКОРТОСТАН
ОБЛ АРХАНГЕЛЬСКАЯ                                   АРХАНГЕЛЬСКАЯ
ХАНТЫ-МАНСИЙСКИЙ АО                              ХАНТЫ-МАНСИЙСКИЙ
РЕСП БАШКОРТОСТАН                                    БАШКОРТОСТАН
ПЕРМСКИЙ К

In [16]:
# заменяем исходные категории переменной living_region на новые
data['living_region'] = data['living_region'].map(regions)

In [17]:
# смотрим уникальные значения по переменной living_region
data['living_region'].unique()

array(['КРАСНОДАРСКИЙ', 'МОСКВА', 'САРАТОВСКАЯ', 'ВОЛГОГРАДСКАЯ',
       'ЧЕЛЯБИНСКАЯ', 'СТАВРОПОЛЬСКИЙ', 'НИЖЕГОРОДСКАЯ', 'МОСКОВСКАЯ',
       'ХАНТЫ-МАНСИЙСКИЙ', 'САНКТ-ПЕТЕРБУРГ', 'БАШКОРТОСТАН',
       'АРХАНГЕЛЬСКАЯ', 'ПЕРМСКИЙ', 'КАРАЧАЕВО-ЧЕРКЕССКАЯ', 'КАЛУЖСКАЯ',
       'ВОЛОГОДСКАЯ', 'РОСТОВСКАЯ', 'УДМУРТСКАЯ', 'ИРКУТСКАЯ',
       'ТЮМЕНСКАЯ', 'БЕЛГОРОДСКАЯ', 'КОСТРОМСКАЯ', 'ХАКАСИЯ', 'ТАТАРСТАН',
       'СВЕРДЛОВСКАЯ', 'ПСКОВСКАЯ', 'ЗАБАЙКАЛЬСКИЙ', 'ОРЕНБУРГСКАЯ',
       'ВОРОНЕЖСКАЯ', 'АСТРАХАНСКАЯ', 'НОВОСИБИРСКАЯ', 'КУРГАНСКАЯ',
       'УЛЬЯНОВСКАЯ', 'МУРМАНСКАЯ', 'КРАСНОЯРСКИЙ', 'БУРЯТИЯ',
       'САХА/ЯКУТИЯ/', 'АМУРСКАЯ', 'ХАБАРОВСКИЙ', 'ЯМАЛО-НЕНЕЦКИЙ',
       'САМАРСКАЯ', 'ТВЕРСКАЯ', 'ЯРОСЛАВСКАЯ', 'ВЛАДИМИРСКАЯ',
       'ЛЕНИНГРАДСКАЯ', 'ОРЛОВСКАЯ', 'КЕМЕРОВСКАЯ', 'ОМСКАЯ', 'ЧЕЧЕНСКАЯ',
       'КУРСКАЯ', 'ТУЛЬСКАЯ', 'АДЫГЕЯ', 'КОМИ', 'ПРИМОРСКИЙ',
       'СМОЛЕНСКАЯ', 'КИРОВСКАЯ', 'ДАГЕСТАН', 'ПЕНЗЕНСКАЯ', 'КАРЕЛИЯ',
       'ТОМСКАЯ', 'МАГАДАНСКАЯ', 'МАРИЙЭЛ', 'ИВАНО

In [18]:
# смотрим количество уникальных значений переменной living_region
data['living_region'].nunique()

85

__ПРЕОБРАЗОВАНИЕ ТИПОВ ПЕРЕМЕННЫХ__


In [19]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  object 
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  float64
 7   score_shk             170739 non-null  object 
 8   education             170741 non-null  object 
 9   living_region         170746 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 18.

In [20]:
# категориальные переменные tariff_id и open_account_flg неверно записаны как количественные
# преобразуем их в тип object
for i in ['tariff_id', 'open_account_flg']:
    data[i] = data[i].astype('object')

In [21]:
# количественные переменные credit_sum и score_shk не верно записаны как категориальные переменные
# преобразуем их в тип float 
for i in ['credit_sum', 'score_shk']:
    data[i] = data[i].str.replace(',', '.').astype('float')

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job_position          170746 non-null  object 
 4   credit_sum            170744 non-null  float64
 5   credit_month          170746 non-null  int64  
 6   tariff_id             170746 non-null  object 
 7   score_shk             170739 non-null  float64
 8   education             170741 non-null  object 
 9   living_region         170746 non-null  object 
 10  monthly_income        170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  open_account_flg      170746 non-null  object 
dtypes: float64(6), int64(1), object(7)
memory usage: 18.

In [23]:
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


__ПЕРЕИМЕНОВАНИЕ КАТЕГОРИЙ ПЕРЕМЕННЫХ__

In [24]:
# изменим значения переменной gender 
# создаем словарь, в котором ключем является старое название категории, значением - новое название категории
d = {'M': 'Male', 'F': 'Female'}
# передаем словарь в метод map 
data['gender'] = data['gender'].map(d)
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,Male,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,Female,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,Male,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,Female,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,Male,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


In [25]:
# аналогичную операцию можно выполнить с помощью метода replace()
# создаем словарь, в котором ключем является старое название категории, значением - новое название категории
f = {'F': 'Female', 'M': 'Male'}
# передаем в метод replace словарь
data = data.replace({'gender': f})
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,Male,NaN,NaN,UMN,59998.00,10,1.6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,Female,NaN,MAR,UMN,10889.00,6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,Male,32.0,MAR,SPC,10728.00,12,1.1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,Female,27.0,NaN,SPC,12009.09,12,1.1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,Male,45.0,NaN,SPC,NaN,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


__ОБРАБОТКА РЕДКИХ КАТЕГОРИЙ__


Редкие категории являются источником шума в данных, который негативно повлияет на качество модели. Кроме того, при разбиении набора на обучающую и контрольную выборки может оказаться, что данная категория отсутствует в обучающей выборке, но присутствует в контрольной. Это вызовет проблемы при моделировании 

In [26]:
# выводим частоты категорий по каждой категориальной переменной
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
for c in categorical_columns:
    print(data[c].value_counts())

gender
Female    88697
Male      82049
Name: count, dtype: int64
marital_status
MAR    93954
UNM    52149
DIV    16969
CIV     4196
WID     3475
Name: count, dtype: int64
job_position
SPC    134680
UMN     17674
BIS      5591
PNA      4107
DIR      3750
ATP      2791
WRK       656
NOR       537
WOI       352
INP       241
BIU       126
WRP       110
PNI        65
PNV        40
PNS        12
HSK         8
INV         5
ONB         1
Name: count, dtype: int64
tariff_id
1.1     69355
1.6     39117
1.32    15537
1.4     10970
1.5      7497
1.9      5538
1.43     3930
1.3      3339
1.16     3232
1.0      2245
1.44     2228
1.19     2102
1.2      1306
1.7      1007
1.17      717
1.21      579
1.94      414
1.22      376
1.23      370
1.91      317
1.24      303
1.41      132
1.25       56
1.18       36
1.26       11
1.28       10
1.52        7
1.27        6
1.48        5
1.56        2
1.96        1
1.29        1
Name: count, dtype: int64
education
SCH    87537
GRD    72591
UGR     9941
PGR  

In [27]:
# job_position, tariff_id, living_region содержат множественные редкие категории
# выводим частоты категорий для переменной job_position 
# dropna = False выведет частоту пропусков, если они есть
print(data['job_position'].value_counts(dropna = False))

job_position
SPC    134680
UMN     17674
BIS      5591
PNA      4107
DIR      3750
ATP      2791
WRK       656
NOR       537
WOI       352
INP       241
BIU       126
WRP       110
PNI        65
PNV        40
PNS        12
HSK         8
INV         5
ONB         1
Name: count, dtype: int64


In [28]:
# видим, что переменная не содержит пропусков. Все категории с частотой менее 55 наблюдений объеденим в отдельную
# категорию other 
data['job_position'] = data['job_position'].replace(['PNV', 'PNS', 'HSK', 'INV', 'ONB'], 'OTHER')

In [29]:
print(data['job_position'].value_counts(dropna = False))

job_position
SPC      134680
UMN       17674
BIS        5591
PNA        4107
DIR        3750
ATP        2791
WRK         656
NOR         537
WOI         352
INP         241
BIU         126
WRP         110
OTHER        66
PNI          65
Name: count, dtype: int64


In [30]:
# теперь выведем частоты категорий по переменной tariff_id
print(data['tariff_id'].value_counts(dropna = False))

tariff_id
1.1     69355
1.6     39117
1.32    15537
1.4     10970
1.5      7497
1.9      5538
1.43     3930
1.3      3339
1.16     3232
1.0      2245
1.44     2228
1.19     2102
1.2      1306
1.7      1007
1.17      717
1.21      579
1.94      414
1.22      376
1.23      370
1.91      317
1.24      303
1.41      132
1.25       56
1.18       36
1.26       11
1.28       10
1.52        7
1.27        6
1.48        5
1.56        2
1.96        1
1.29        1
Name: count, dtype: int64


In [31]:
# Все категории с частотой менее 55 наблюдений объеденим в отдельную категорию 1.99
# кроме того на основе категориальной переменной tariff_id создадим количественную переменную tariff
# затем переменной tariff_id присвоим тип str и заменим в ее значениях точки на нижнее подчеркивание
# если этого не сделать, то при использовании Catboost выдаст ошибку

data.loc[data['tariff_id'].value_counts()[data['tariff_id']].values < 55, 'tariff_id'] = 1.99
data['tariff'] = data['tariff_id'].astype('float')
data['tariff_id'] = data['tariff_id'].astype('str').str.replace('.', '_')

# этот программный код будет работать только при отсутствии Nan / NA, в противном случае будет ошибка

In [32]:
print(data['tariff_id'].value_counts(dropna = False))

tariff_id
1_1     69355
1_6     39117
1_32    15537
1_4     10970
1_5      7497
1_9      5538
1_43     3930
1_3      3339
1_16     3232
1_0      2245
1_44     2228
1_19     2102
1_2      1306
1_7      1007
1_17      717
1_21      579
1_94      414
1_22      376
1_23      370
1_91      317
1_24      303
1_41      132
1_99       79
1_25       56
Name: count, dtype: int64


In [33]:
# выведем частоты переменной living_region. Поскольку категорий очень много, выведем последние 10
print(data['living_region'].value_counts(dropna = False).tail(10))

living_region
ЕВРЕЙСКАЯ      203
nan            192
НЕНЕЦКИЙ       172
МАГАДАНСКАЯ    159
ДАГЕСТАН        69
АЛТАЙ           54
ЧУКОТСКИЙ       32
ЧЕЧЕНСКАЯ       31
ИНГУШЕТИЯ       19
ЧИТИНСКАЯ       17
Name: count, dtype: int64


Теперь необходимо определится с пороговой относительной частотой. В данном случае мы хотим объединить категории с частотой 50 наблюдений и меньше в категорию other. мы делим 50 наблюдений на общее количество наблюдений в наборе данных (170 746 наблюдений), умножаем на 100 и получаем пороговую относительную частоту 0,029. С помощью программного кода, приведенного ниже, мы делим частоту каждой категории на общее количество наблюдений на общее количество наблюдений в наборе данных умножаем на 100 и получаем пороговую относительную частоту 0,029. Если относительная частота категории меньше 0,029, возвращаем значение True если больше возвращаем False

In [34]:
region_series = data['living_region'].value_counts()
mask = (region_series/region_series.sum() * 100).lt(0.029)
mask.tail(10)

living_region
ЕВРЕЙСКАЯ      False
nan            False
НЕНЕЦКИЙ       False
МАГАДАНСКАЯ    False
ДАГЕСТАН       False
АЛТАЙ          False
ЧУКОТСКИЙ       True
ЧЕЧЕНСКАЯ       True
ИНГУШЕТИЯ       True
ЧИТИНСКАЯ       True
Name: count, dtype: bool

In [35]:
# теперь с помощью функции np.where() мы все категории по которым получили TRUE заменим на категорию OTHER 
data['living_region'] = np.where(data['living_region'].isin(region_series[mask].index), 'OTHER', data['living_region'])
print(data['living_region'].value_counts(dropna = False).tail(10))

living_region
КАМЧАТСКИЙ               412
СЕВЕРНАЯОСЕТИЯ-АЛАНИЯ    379
КАЛМЫКИЯ                 305
ЕВРЕЙСКАЯ                203
nan                      192
НЕНЕЦКИЙ                 172
МАГАДАНСКАЯ              159
OTHER                     99
ДАГЕСТАН                  69
АЛТАЙ                     54
Name: count, dtype: int64


__РАЗБИЕНИЕ НАБОРА НА ОБУЧАЮЩУЮ И КОНТРОЛЬНУЮ ВЫБОРКИ__

In [36]:
# с помощью метода .sample() из исходного датафрейма data случайно отбираем 70% наблюдений в обучающий датафрейм test
train = data.sample(frac = 0.7, random_state = 200)

# в исходном датафрейме оставляем только те наблюдения, индексные метки которых отличаются от индекса наблюдений,
# попавших в обучающий датафрейм train и записываем контрольный датафрейм test
test = data.drop(train.index)

__ИМПУТАЦИЯ ПРОПУСКОВ__


In [37]:
# выводим информацию о количестве пропусков по каждой переменной в обучающей выборке
train.isnull().sum()

gender                     0
age                        1
marital_status             1
job_position               0
credit_sum                 1
credit_month               0
tariff_id                  0
score_shk                  4
education                  2
living_region              0
monthly_income             2
credit_count            6477
overdue_credit_count    6477
open_account_flg           0
tariff                     0
dtype: int64

In [38]:
# выводим информацию о количестве пропусков по каждой переменной в контрольной выборке
test.isnull().sum()

gender                     0
age                        2
marital_status             2
job_position               0
credit_sum                 1
credit_month               0
tariff_id                  0
score_shk                  3
education                  3
living_region              0
monthly_income             3
credit_count            2753
overdue_credit_count    2753
open_account_flg           0
tariff                     0
dtype: int64

Импутация __средним, медианой и прочими статистиками__ необходимо выполнять после разбиения на обучающую и контрольную выборки

Пропуски можно заменить __константой__ до разбиения на выборки

Бинарные переменные, у которых есть пропуски, можно превратить в __категориальные__ с тремя признакими -1, 0, 1

Пропуски в категориальных переменных можно заменить самой часто встречающейся категорией - __модой__



In [40]:
# заполняем пропуски в переменной age медианой
train['age'].fillna(train['age'].median(), inplace = True)
test['age'].fillna(train['age'].median(), inplace = True)

# заполняем пропуски в переменной age средним
# train['age'].fillna(train['age'].mean(), inplace = True)
# test['age'].fillna(train['age'].mean(), inplace = True)

In [41]:
# заполняем пропуски в переменных credit_sum, score_shk, monthly_income медианами
for i in ['credit_sum', 'score_shk', 'monthly_income']:
    train[i].fillna(train[i].median(), inplace = True)
    test[i].fillna(train[i].median(), inplace = True)

In [42]:
# заполняем пропуски в переменных credit_count, overdue_credit_count -1
for i in ['credit_count', 'overdue_credit_count']:
    train[i].fillna(-1, inplace = True)
for i in ['credit_count', 'overdue_credit_count']:
    test[i].fillna(-1, inplace = True)

In [44]:
# выводим частоты категорий для переменной marital_status
print(train['marital_status'].value_counts(dropna = False))
print(test['marital_status'].value_counts(dropna = False))

marital_status
MAR    65652
UNM    36581
DIV    11918
CIV     2942
WID     2428
NaN        1
Name: count, dtype: int64
marital_status
MAR    28302
UNM    15568
DIV     5051
CIV     1254
WID     1047
NaN        2
Name: count, dtype: int64


Заменим пропуски модой - самой часто встречающейся категорией, в данном случае категорией MAR

In [45]:
# вычисляем моду для переменной 
train['marital_status'].mode()

0    MAR
Name: marital_status, dtype: object

In [46]:
# выполняем импутацию пропусков модой
train['marital_status'] = train['marital_status'].fillna('MAR')
test['marital_status'] = test['marital_status'].fillna('MAR')

In [47]:
# выводим частоты категорий для переменной marital_status
print(train['marital_status'].value_counts(dropna = False))
print(test['marital_status'].value_counts(dropna = False))

marital_status
MAR    65653
UNM    36581
DIV    11918
CIV     2942
WID     2428
Name: count, dtype: int64
marital_status
MAR    28304
UNM    15568
DIV     5051
CIV     1254
WID     1047
Name: count, dtype: int64


In [48]:
# выводим частоты категорий для переменной education
print(train['education'].value_counts(dropna = False))
print(test['education'].value_counts(dropna = False))

education
SCH    61126
GRD    50928
UGR     6973
PGR      418
ACD       75
NaN        2
Name: count, dtype: int64
education
SCH    26411
GRD    21663
UGR     2968
PGR      147
ACD       32
NaN        3
Name: count, dtype: int64


In [49]:
# выполняем импутацию пропусков модой
train['education'].fillna(train['education'].value_counts().index[0], inplace = True)
test['education'].fillna(train['education'].value_counts().index[0], inplace = True)

In [50]:
# выводим частоты категорий для переменной education
print(train['education'].value_counts(dropna = False))
print(test['education'].value_counts(dropna = False))

education
SCH    61128
GRD    50928
UGR     6973
PGR      418
ACD       75
Name: count, dtype: int64
education
SCH    26414
GRD    21663
UGR     2968
PGR      147
ACD       32
Name: count, dtype: int64


__КОНСТРУИРОВАНИЕ НОВЫХ ПРИЗНАКОВ__ 

In [51]:
# создаем переменную paym, которая является отношением выданной суммы кредита (credit_sum) к сроку кредита (credit_month)
# то есть ежемесячной суммой кредита
train['paym'] = train['credit_sum'] / train['credit_month']
test['paym'] = test['credit_sum'] / test['credit_month']

In [52]:
# создаем переменную pti, которая является отношением ежемесячной суммы кредита (paym) к ежемесячному заработку (monthly_income)
train['pti'] = train['paym'] / train['monthly_income']
test['pti'] = test['paym'] / test['monthly_income']

In [55]:
# заменяем бесконечные значения на 1
train['pti'].replace([np.inf, -np.inf], 1, inplace = True)
test['pti'].replace([np.inf, -np.inf], 1, inplace = True)

__Создание переменной у которой значения основаны на значениях исходной переменной__

In [56]:
# выводим уникальные значения исходной переменной job_position 
print(train['job_position'].unique())

['SPC' 'UMN' 'BIS' 'DIR' 'PNA' 'ATP' 'OTHER' 'WRK' 'BIU' 'NOR' 'WOI' 'INP'
 'PNI' 'WRP']


In [57]:
# создаем словарь в котором ключем будет значение исходной переменной job_position а значением - значение будущей 
# переменной avzarplata

dct = {'UMN': 51000, 'SPC': 63000, 'INP': 55000, 'DIR': 60000,
       'ATP': 46000, 'PNA': 71000, 'BIS': 86000, 'WOI': 76000,
       'NOR': 54000, 'WRK': 77000, 'WRP': 75000, 'PNV': 67000,
       'BIU': 43000, 'PNI': 69000, 'HSK': 74000, 'PNS': 44000,
       'INV': 88000, 'ONB': 62000, 'OTHER': 20000}

# создаем новую переменную avzarplata, у которой значения сопоставлены значениями переменной job_position 
train['avzarplata'] = train['job_position'].map(dct)
train.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg,tariff,paym,pti,avzarplata
53397,Male,28.0,UNM,SPC,33579.0,10,1_4,0.431360,SCH,АМУРСКАЯ,36000.0,2.0,0.0,0,1.40,3357.900000,0.093275,63000
143962,Male,38.0,UNM,SPC,23511.0,10,1_32,0.358472,GRD,СТАВРОПОЛЬСКИЙ,45000.0,2.0,0.0,0,1.32,2351.100000,0.052247,63000
146922,Female,25.0,MAR,SPC,39990.0,24,1_5,0.613475,GRD,РОСТОВСКАЯ,50000.0,6.0,0.0,0,1.50,1666.250000,0.033325,63000
63697,Female,65.0,UNM,UMN,3490.0,6,1_3,0.371355,GRD,ХАКАСИЯ,35000.0,4.0,0.0,0,1.30,581.666667,0.016619,51000
54503,Male,49.0,MAR,BIS,36358.0,10,1_6,0.645187,SCH,ЧЕЛЯБИНСКАЯ,50000.0,2.0,0.0,0,1.60,3635.800000,0.072716,86000


In [60]:
# удалим переменную avzarplata
train.drop('avzarplata', axis = 1, inplace = True)
train.head()

KeyError: "['avzarplata'] not found in axis"